## 最小ID

In [1]:
using StatsBase
## 20万个id
n=200_001
ids=sample(1:n,n-1,replace=false);

In [2]:
## for嵌套for
function min_id_for_and_for(x::Array{Int64})::Int64
    for i = 1:length(x)
        not_has_id = true
        for j =1:length(x)
            if i == x[j]
                not_has_id =false
            end
        end
        if not_has_id
            return i
        end
    end
    return length(x)
end

## for+in
function min_id_for_and_in(x::Array{Int64})::Int64
    for i = 1:length(x)
        if i in x
            continue
        else
            return i
        end
    end
    return length(x)
end

## 书中方法改进一
function min_id_change_1(x::Array{Int64})::Int64
    myfalse = trues(length(x))
    for i in x
        if i <= length(x)
            myfalse[i]=false
        end
    end
    for i=1:length(myfalse)
        if myfalse[i]
            return i
        end
    end
    return length(x)+1
end


## 书中方法改进二
function min_id_change_2(x,l,u)::Int64
    n = length(x)    
    while n > 0

        middle = floor((l+u)/2)
        left=l   
        for right = l:n+l-1
            if x[right] <= middle
                tmp = x[right]
                x[right] = x[left]
                x[left] = tmp
                left = left +1
            end
        end
        if left == middle+1
            n = n - left + l
            l = left
        else
            u = left
            n = left-l
        end
    end
    return l       
end
    

min_id_change_2 (generic function with 1 method)

In [3]:
## 使用for嵌套for
@time min_id_for_and_for(ids)
@time min_id_for_and_for(ids)

  2.275260 seconds (19.06 k allocations: 963.168 KiB)
  2.377084 seconds (5 allocations: 176 bytes)


11217

In [4]:
## 使用for+in
@time min_id_for_and_in(ids)
@time min_id_for_and_in(ids)

  0.579609 seconds (16.72 k allocations: 844.855 KiB)
  0.554984 seconds (5 allocations: 176 bytes)


11217

In [5]:
## 改进一方法
@time min_id_change_1(ids)
@time min_id_change_1(ids)

  0.040999 seconds (49.59 k allocations: 2.455 MiB, 16.35% gc time)
  0.000401 seconds (8 allocations: 24.719 KiB)


11217

In [6]:
## 改进二方法
@time min_id_change_2(ids,1,length(ids))
@time min_id_change_2(ids,1,length(ids))

  0.030979 seconds (29.76 k allocations: 1.539 MiB)
  0.001301 seconds (6 allocations: 192 bytes)


11217

### julia，python结果比较

**python结果**

- for+for:min_id() cost:101.6906300 s
- for+in:min_id_with_not_in() cost:0.4106679 s!
- change1:min_id_change_1() cost:0.0708082 s!
- change2+递归:min_id_change_2() cost:36.0500872 s!
- change2+迭代:min_id_change_2_iteration() cost:0.7271988 s!

**julia结果**

- for+for:  cost: 1.057379s
- for+in:  cost: 0.272523s!
- change1:  cost: 0.000660s!
- change2+迭代:  cost:0.001244 s!

而且当经过一边运行后，第二遍在运行，速度提升很大。

而且能很明显的看出，内存的消耗都小了很多，能看出不同方法提升差异，以及`julia`在性能上的提升。

## 丑数

In [7]:
## 暴力方法，其实这个方法和python的很像，但是刚学julia，有很多还是不熟悉，所以重复写一点。

function ugly_number_in_for(number::Int64)::Int64
    count = 0
    n=0
    while count<number
        n += 1
        if is_ugly(n)
            count += 1
        end
    end
    return n
end


function is_ugly(number::Int64)::Bool
    while mod(number,2)==0
        number = number / 2
    end
    while mod(number,3)==0
        number = number/3
    end
    while mod(number,5) ==0
        number = number/5
    end
    if number==1
        return true
    else
        return false
    end
end

## queue

function ugly_number_in_queue(number::Int64)::Int64
    count=1
    queue = [1]
    has_disp = []
    while count<number
        min_n = popfirst!(queue)
        if !(min_n*2 in has_disp)
            push!(has_disp,min_n*2)
            push!(queue,min_n*2)
        end
        if !(min_n*3 in has_disp)
            push!(has_disp,min_n*3)
            push!(queue,min_n*3)
        end
        if !(min_n*5 in has_disp)
            push!(has_disp,min_n*5)
            push!(queue,min_n*5)
        end
        count += 1
        queue = sort(unique(queue))
    end
    return queue[1]
end
            

## different queue

function ugly_number_in_dif_queue(number::Int64)::Int64
    count=1
    queue2=[2]
    queue3=[3]
    queue5=[5]
    min_n = 1
    while count < number
        min_n = minimum([queue2[1],queue3[1],queue5[1]])
        if min_n == queue2[1]
            popfirst!(queue2)
            push!(queue2,2*min_n)
            push!(queue3,3*min_n)
            push!(queue5,5*min_n)
        elseif min_n ==queue3[1]
            popfirst!(queue3)
            push!(queue3,3*min_n)
            push!(queue5,5*min_n)
        else
            popfirst!(queue5)
            push!(queue5,5*min_n)
        end
        count += 1
    end
    return min_n
end           
    
        
        

ugly_number_in_dif_queue (generic function with 1 method)

In [8]:
@time ugly_number_in_for(500)
@time ugly_number_in_for(500)

  0.159636 seconds (69.74 k allocations: 3.600 MiB)
  0.127385 seconds (5 allocations: 176 bytes)


937500

In [9]:
@time ugly_number_in_queue(500)
@time ugly_number_in_queue(500)

  0.116274 seconds (684.21 k allocations: 21.278 MiB, 3.59% gc time)
  0.031030 seconds (483.35 k allocations: 11.449 MiB, 10.38% gc time)


937500

In [10]:
@time ugly_number_in_dif_queue(500)
@time ugly_number_in_dif_queue(500)

  0.058132 seconds (125.27 k allocations: 6.155 MiB)
  0.000049 seconds (1.83 k allocations: 85.016 KiB)


937500

### julia，python结果比较

**python结果**

- 暴力方法:0.56 seconds
- 一个队列：0.023 seconds
- 多个队列：0.000409 seconds

**julia结果**

- 暴力方法:0.122705 seconds
- 一个队列：0.020103 seconds
- 多个队列：0.000072 seconds


julia的结果都是用的第二次运行，目前不清楚，不是写到这样的交互式中，是不是只要运行过一次就可以了。